In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

from scipy import stats

from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
# from sklearn.pipeline import make_pipeline
# from sklearn.pipeline import Pipeline

In [2]:
carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [3]:
carseats_df = carseats.data

In [8]:
carseats_df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,Urban_Yes,US_Yes
0,9.50,138,73,11,276,120,42,17,False,False,True,True
1,11.22,111,48,16,260,83,65,10,True,False,True,True
2,10.06,113,35,10,269,80,59,12,False,True,True,True
3,7.40,117,100,4,466,97,55,14,False,True,True,True
4,4.15,141,64,3,340,128,38,13,False,False,True,False


In [4]:
carseats_df = pd.get_dummies(carseats_df, columns = ["ShelveLoc", "Urban", "US"], drop_first=True)

In [5]:
X2, y = carseats_df.iloc[:, carseats_df.columns.isin(["Advertising", "Price"])], carseats_df["Sales"]

In [7]:
x2 = np.array(X2)
lm2 = LinearRegression()
lm2.fit(x2, y)
lm2.score(x2, y)

0.28185560334302273

In [9]:
X3, y = carseats_df.iloc[:, carseats_df.columns != "Sales"], carseats_df["Sales"]

In [10]:
x3 = np.array(X3)
lm3 = LinearRegression()
lm3.fit(x3, y)
lm3.score(x3, y)

0.8734133434127013